# multistep regression classifier with stochasticity

Jingmeng

the general idea to solve the multiple output problem here is that we first predict one output, then use the output of that as an input and predict the other output(s)

an alternative to this is that we make a (very small) neural network which inherently supports (correlated) vector output

In [5]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import TimeSeriesSplit

df = pd.read_csv("F.txt", sep="\t", header=None)
dfV1 = pd.DataFrame(df.iloc[:,0])
dfV1 = dfV1.rename(columns={0:"v1"})

print(dfV1)

      v1
0      0
1      0
2      0
3      0
4      0
...   ..
3819   0
3820   0
3821   0
3822   0
3823   0

[3824 rows x 1 columns]


In [7]:
dfV1["v1_octave"] = dfV1["v1"] //12
dfV1["v1_pitch_class"] = dfV1["v1"] % 12
dfV1

,v1,v1_octave,v1_pitch_class
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
3819,0,0,0
3820,0,0,0
3821,0,0,0
3822,0,0,0


In [8]:
for i in range(1,30):
    dfV1["v1_octave_lag"+str(i)] = dfV1["v1_octave"].shift(i).fillna(0).astype(int)
    dfV1["v1_pitch_class_lag"+str(i)] = dfV1["v1_pitch_class"].shift(i).fillna(0).astype(int)

In [9]:
dfV1

,v1,v1_octave,v1_pitch_class,v1_octave_lag1,v1_pitch_class_lag1,v1_octave_lag2,v1_pitch_class_lag2,v1_octave_lag3,v1_pitch_class_lag3,v1_octave_lag4,...,v1_octave_lag25,v1_pitch_class_lag25,v1_octave_lag26,v1_pitch_class_lag26,v1_octave_lag27,v1_pitch_class_lag27,v1_octave_lag28,v1_pitch_class_lag28,v1_octave_lag29,v1_pitch_class_lag29
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3819,0,0,0,0,0,0,0,0,0,0,...,5,8,5,8,5,8,5,11,5,11
3820,0,0,0,0,0,0,0,0,0,0,...,5,8,5,8,5,8,5,8,5,11
3821,0,0,0,0,0,0,0,0,0,0,...,0,0,5,8,5,8,5,8,5,8
3822,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,5,8,5,8,5,8


In [16]:
rcv = RidgeClassifierCV(alphas = [0.1, 1., 10.], cv = 10)
rcv.fit(X = dfV1[dfV1.columns.drop(['v1', 'v1_octave', 'v1_pitch_class'])], y = dfV1['v1_octave'])

RidgeClassifierCV(alphas=array([ 0.1,  1. , 10. ]), cv=10)

In [17]:
rcv.alpha_

0.1

In [23]:
rcv_pred = rcv.predict(X = dfV1[dfV1.columns.drop(['v1', 'v1_octave', 'v1_pitch_class'])])
rcv_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [35]:
d = rcv.decision_function(dfV1[dfV1.columns.drop(['v1', 'v1_octave', 'v1_pitch_class'])])
probs = [np.exp(i) / np.sum(np.exp(i)) for i in d]
probs[1:5]


[array([0.69955902, 0.10583804, 0.09672893, 0.09787401]),
 array([0.69955902, 0.10583804, 0.09672893, 0.09787401]),
 array([0.69955902, 0.10583804, 0.09672893, 0.09787401]),
 array([0.69955902, 0.10583804, 0.09672893, 0.09787401])]

In [36]:
rcv.classes_

array([0, 4, 5, 6], dtype=int64)